## **Knapsack problem**

Given a set of items, each with a weight and a value, determine which items to include in the collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

### **Binary programming**

### Sets

* $I$: Set of items, i $\in$ {1,2...,I}

### Parameters
* $\nu_{i}$: Value for item ${i}$, $\forall i \in I$
* $\omega_{i}$: Weight for item ${i}$, $\forall i \in I$
* $W$: Total weight of the knapsack

### Decision variable
$$
     x_{i} =
        \begin{cases}
        1 \space\text{if item ${i}$ is selected,} \space \forall i \in I \\
        0 \space\text{if item ${i}$ is not selected,} \space \forall i \in I
        \end{cases}
$$

### Objective function 

   * **Maximize total value in the knapsack.**

 $$ \ Max\space \ Z = \sum_{i \in I} \nu_{i}*x_{i} $$

 ### Restricciones

 **1. Minimum weight in the knapsack**.

 This constraint makes possible that the collection of items selected is less than or equal to a given limit.

 $$ \sum_{i \in I} \omega_{i}*x_{i} \leq W $$

 **2. Binary space**.

 The decision variable is binary.

 $$ x_{i} \in [0,1], \forall i \in I $$



In [12]:
#Coding the optimal distribution model using Pyomo

#Import the libraries

import numpy as np
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import matplotlib.pyplot as plt

from pyomo.environ import *

In [13]:

# Declare the url for the configuration file
config_file_url = 'C:/Users/j.rodriguez.villegas/OneDrive - Accenture/Coursera/Discrete Optimization/Week 2/configuration_file.xlsx'

# Read the items data from the configuration file
items_raw_data = pd.read_excel(config_file_url, sheet_name ='Items')
items_raw_data.head(26)

,ID,value,weight
0,A,27416,132
1,B,25502,73
2,C,13662,101
3,D,13155,178
4,E,11498,60
5,F,6272,70
6,G,22029,193
7,H,11511,187
8,I,18885,76
9,J,18399,124


In [14]:
items_raw_data.shape

(26, 3)

In [15]:
# Optimization model

items_data = items_raw_data

# Sets
model = pe.ConcreteModel('knapsack_v1')
model.items_knapsack = pe.Set(initialize = items_data['ID'].drop_duplicates())

items_data = items_data.set_index('ID')

print("The number of elements in the set {model.items_knapsack} is: ", len(model.items_knapsack))

The number of elements in the set {model.items_knapsack} is:  26


In [16]:
# Parameters
model.value = pe.Param(model.items_knapsack, initialize = items_data['value'].to_dict())
model.weight = pe.Param(model.items_knapsack, initialize = items_data['weight'].to_dict())
model.max_knapsack_weight = pe.Param(initialize = 500)

In [17]:
# Decision variables
model.x = pe.Var(model.items_knapsack, domain = Binary)

In [18]:
# Objective function
def calculate_total_knapsack_value(model):
    '''
    This function calculates the total value in the knapsack.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.
    
    Return
    ----------
    int
        Total value.
    '''
    total_value = sum(model.value[i] * model.x[i] for i in model.items_knapsack)
    return total_value


In [19]:
# Constraints

def c_1_max_capacity(model):
    '''
    Makes sure the total weight of the items is limited by the knapsack capacity.

    Parameters
    ----------
    model : Pyomo ConcreteModel
        The optimization model.
    
    Returns
    ----------
    Constraint Expression
        Relational expression for the constraint.
    '''
    max_cap_left = sum(model.weight[i] * model.x[i] for i in model.items_knapsack)
    max_cap_right = model.max_knapsack_weight
    max_cap = (max_cap_left <= max_cap_right)
    return max_cap

In [20]:
# Solve the knapsack problem

from pyomo.environ import SolverFactory, SolverStatus, TerminationCondition

model.c_1_max_capacity = pe.Constraint(rule = c_1_max_capacity)
model.obj_funct_calculate_total_knapsack_value = pe.Objective(sense = pe.maximize,  rule = calculate_total_knapsack_value)

solver = pe.SolverFactory('gurobi')
# Set the time limit in seconds
time_limit = 600  # 600 seconds

# Set the solver options, including the time limit
solver.options['TimeLimit'] = time_limit

# Solve the optimization problem
result = solver.solve(model, tee=True, report_timing = True)

# Check the solver's termination condition
termination_condition = result.solver.termination_condition
solver_status = result.solver.status


# Print the results or handle them accordingly
if solver_status == SolverStatus.ok and termination_condition == TerminationCondition.optimal:
    print("Optimal solution found.")
    # Access the solution values, e.g., model.variable_name.value
    # Access the objective value, e.g., model.objective.value
    pe.value(model.obj_funct_calculate_total_knapsack_value)
    # pe.value(model.obj_funct_calculate_total_return_on_investment)
elif solver_status == SolverStatus.ok and termination_condition in [TerminationCondition.maxTimeLimit]:
    print("Time limit reached. No optimal solution found within the specified time.")
else:
    print("No optimal solution found, model is infeasible or unbounded...")
    print("Solver terminated with condition:", solver_status)

        0.00 seconds required to write file
        0.01 seconds required for presolve
Read LP format model from file C:\Users\JRODRI~1.VIL\AppData\Local\Temp\tmp4wqfrz39.pyomo.lp
Reading time = 0.01 seconds
x27: 2 rows, 27 columns, 27 nonzeros
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 27 columns and 27 nonzeros
Model fingerprint: 0x80803120
Variable types: 1 continuous, 26 integer (26 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [6e+03, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 80268.000000
Presolve removed 2 rows and 27 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iteratio

In [21]:
print(pe.value(model.obj_funct_calculate_total_knapsack_value))

144157.0


In [22]:
model.x.pprint()

x : Size=26, Index=items_knapsack
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :   0.0 :     1 : False : False : Binary
      B :     0 :   1.0 :     1 : False : False : Binary
      C :     0 :   0.0 :     1 : False : False : Binary
      D :     0 :   0.0 :     1 : False : False : Binary
      E :     0 :   0.0 :     1 : False : False : Binary
      F :     0 :   0.0 :     1 : False : False : Binary
      G :     0 :   0.0 :     1 : False : False : Binary
      H :     0 :   0.0 :     1 : False : False : Binary
      I :     0 :   1.0 :     1 : False : False : Binary
      J :     0 :   0.0 :     1 : False : False : Binary
      K :     0 :   0.0 :     1 : False : False : Binary
      L :     0 :   0.0 :     1 : False : False : Binary
      M :     0 :   0.0 :     1 : False : False : Binary
      N :     0 :   0.0 :     1 : False : False : Binary
      O :     0 :   0.0 :     1 : False : False : Binary
      P :     0 :   0.0 :     1 : False : False : Bina